In [1]:
!pip install transformers
!pip install PersianStemmer
!pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade
!pip install hazm
!pip install demoji


     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 3.3 MB 46.6 MB/s 
     |████████████████████████████████| 636 kB 44.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 810 kB 5.0 MB/s 
  Created wheel for PersianStemmer: filename=PersianStemmer-1.0.0-py3-none-any.whl size=810281 sha256=c1f5a08ff34ec38c304a2b9982c95bfc9f07e4fa77a28f94b5fd34a597ab82f8
  Stored in directory: /root/.cache/pip/wheels/d2/f7/96/55f0070725e6ce1191aa9a931e962fc0f551b68ed919fb7f8e
  Created wheel for patricia-trie: filename=patricia_trie-10-py3-none-any.whl size=7768 sha256=380f1cddda931ab51879ed1722cf4eeda7cb78cbdd3b7555f676d26da9f605a8
  Stored in directory: /root/.cache/pip/wheels/32/89/b4/46456cda461ffee3f6ef09a68bccb5d175b65ff2b922b5b5b8
Successfully built Per

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0, '/content/drive/My Drive/modules')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import preprocess
import save_dataframe
from statistics import mean
import demoji
demoji.download_codes()

... OK (Got response in 0.17 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [4]:
def run_svm():
    SVM_model.fit(x_train, y_train)
    prediction_SVM = SVM_model.predict(x_test)
    f_score = round(((f1_score(y_test, prediction_SVM)) * 100), 2)
    accuracy = round((accuracy_score(y_test, prediction_SVM) * 100), 2)
    precision = round((precision_score(y_test, prediction_SVM) * 100), 2)
    recall = round((recall_score(y_test, prediction_SVM) * 100), 2)
    return f_score, accuracy, precision, recall

In [5]:
stop_words = 'custom'
stemming = True
dataset = preprocess.auto_preprocess(pd.read_csv('/content/drive/MyDrive/data/gold_dataset.csv'), stop_words_type=stop_words, stemming=stemming)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

custom stop words selected



there is no outlier in data
auto preprocess completed done on dataframe!
Enjoy!


In [6]:
vectorizer = CountVectorizer(ngram_range=(1,3), min_df=4, max_df=0.8)
x = vectorizer.fit_transform(dataset['sentence'])
y = (dataset['label'].to_numpy())
SVM_model = svm.SVC(random_state=42)
params = {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVM_model.set_params(**params)

f1_list = []
accuracy_list = []
precision_list = []
recall_list = []

for train_index, test_index in skf.split(dataset['sentence'], dataset['label']):

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    f1, accuracy, precision, recall = run_svm()
    f1_list.append(round(f1, 2))
    accuracy_list.append(round(accuracy, 2))
    precision_list.append(round(precision, 2))
    recall_list.append(round(recall, 2))

print (f"f1 is: {mean(f1_list)}\naccuracy is: {mean(accuracy_list)}\nprecision is: {mean(precision_list)}\nrecall is: {mean(recall_list)}")     


f1 is: 92.592
accuracy is: 92.708
precision is: 93.998
recall is: 91.252
